The idea is to model the impact of fishing quotas on a fish population, while taking into account food availability

Import all useful libraries

In [ ]:
#%load_ext autoreload
#%autoreload 2
import numpy as np
import copy 
import random
import pandas as pd
from tqdm import tqdm
from scipy.stats import binom
from scipy.optimize import minimize
import matplotlib.pyplot as plt

Import the q-learning function

In [ ]:
from robust_q_learning import *

First simple settings

In [ ]:
X = np.array([(0,0), (0,5), (0,10), (5,0), (5,5), (5,10), (10,0), (10,5), (10,10)]) # States
A = np.array([0, 5, 10]) # Actions

def r(x,a,y):
    x1, x2 = x
    y1, y2 = y
    return(a * (x2 - a >= 0))# + (y1 + y2 > 0)) # Reward function

def P(x,a):
    x1, x2 = x
    if a==0:
        return (min(x2, 10), min(x1 + x2, 10))
    elif a==5:
        return (max(x2 - 5, 0), min(x1 + max(x2 - 5, 0), 10))
    elif a==10:
        return (0, x1)

alpha      = 0.95  # Discount Factor
x_0        = (5,5) # Initial Value
k_0        = 0     # Initial index of the corresponding MDP, starting with the central proba of 1/2
eps_greedy = 0.1   # Epsilon greedy policy

In [ ]:
Nr_iter = 100_000

Q_opt_robust, Gamma = robust_q_learning(X, A, r, np.array([P]), alpha, x_0, k_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))

In [ ]:
# Build the functions that allow us to get the index of an element a (reps. x) in A (resp. X)
if np.ndim(A) > 1:
    A_list = A
else:
    A_list = np.array([[a] for a in A])
if np.ndim(X) > 1:
    X_list = X
else:
    X_list = np.array([[x] for x in X])

def a_index(a):
    return np.flatnonzero((a==A_list).all(1))[0]
def x_index(x):
    return np.flatnonzero((x==X_list).all(1))[0]

# Get the result of the Q-Learning algorithm,
# Get the optimal results for each x in X
def a_opt(x, Q_opt):
    return A[np.argmax(Q_opt[x_index(x),:])]

In [ ]:
df = pd.DataFrame(np.array([[a_opt(x, Q_opt_robust) for x in X]]))
df.columns = ['(0,0)', '(0,5)', '(0,10)', '(5,0)', '(5,5)', '(5,10)', '(10,0)', '(10,5)', '(10,10)']
df

In [ ]:
plt.plot(Gamma)